<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc">
    <ul class="toc-item">
        <li>
            <span>
                <a href="#Изучение-данных-из-файла" data-toc-modified-id="Изучение-данных-из-файла-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Изучение данных из файла</a>
            </span>
        </li>
            <li>
            <span>
                <a href="#Разделение-исходных-данных-на-выборки" data-toc-modified-id="Разделение-исходных-данных-на-выборки-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Разделение исходных данных на выборки</a></span>
        </li>
            <li>
            <span>
                <a href="#Выбор-лучшей-модели" data-toc-modified-id="Выбор-лучшей-модели-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выбор лучшей модели</a>
            </span>
        </li>
            <li>
            <span>
                <a href="#Проверка-качества-и-вменяемости-выбранной-модели" data-toc-modified-id="Проверка-качества-и-вменяемости-выбранной-модели-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Проверка качества и вменяемости выбранной модели</a>
            </span>
        </li>
            <li>
             <a href="#Общий-вывод" data-toc-modified-id="Общий-вывод-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Общий вывод</a></span>
    
 </div>

# Рекомендация тарифов
Оператор мобильной связи «Мегалайн» выяснил: многие клиенты пользуются архивными тарифами. Они хотят построить систему, способную проанализировать поведение клиентов и предложить пользователям новый тариф: «Смарт» или «Ультра».
В нашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы. Нужно построить модель для задачи классификации, которая выберет подходящий тариф.

Описание данных  
Каждый объект в наборе данных — это информация о поведении одного пользователя за месяц. Известно:  
сalls — количество звонков,  
minutes — суммарная длительность звонков в минутах,  
messages — количество sms-сообщений,  
mb_used — израсходованный интернет-трафик в Мб,  
is_ultra — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).

Для выполнения задачи необходимо сравнить несколько моделей классификации и сделать выбор в пользу обладающей наибольшим параметром accuracy, то есть долей правильных ответов.

## Изучение данных из файла

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.dummy import DummyClassifier

In [2]:
try:
    df = pd.read_csv('users_behavior.csv')
except:
    df = pd.read_csv('/datasets/users_behavior.csv')

У нас есть информация о том, что данные уже предобработаны, поэтому просто быстро осмотрим датасет.

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [4]:
df.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


In [5]:
df.describe()

,calls,minutes,messages,mb_used,is_ultra
count,3214.000000,3214.000000,3214.000000,3214.000000,3214.000000
mean,63.038892,438.208787,38.281269,17207.673836,0.306472
std,33.236368,234.569872,36.148326,7570.968246,0.461100
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,40.000000,274.575000,9.000000,12491.902500,0.000000
50%,62.000000,430.600000,30.000000,16943.235000,0.000000
75%,82.000000,571.927500,57.000000,21424.700000,1.000000
max,244.000000,1632.060000,224.000000,49745.730000,1.000000


In [6]:
df['is_ultra'].value_counts(normalize=True)

0    0.693528
1    0.306472
Name: is_ultra, dtype: float64

Пропусков нет.  
Форматы данных подхотят для наших задач.  
Названия столбцов соответствуют хорошему стилю.  
Данные по целевому показателю 'is_ultra', не сбалансированы, можно сделать вывод, что необходимо построить модель которая покажет результат выше 70%, которые можно получить просто заполнив столбец доминирующим значением.

## Разделение исходных данных на выборки

Для начала разобьем датасет на признаки, целевым признаком будут данные из столбца "is_ultra".

In [7]:
features = df.drop('is_ultra', axis=1)
target = df['is_ultra']

Теперь данные надо разделить на три выборки: обучающую, валидационную и тестовую.  
Общепринятые пропорции - 3:1:1(Обучающая:валидационная:тесмтовая).  
Проще всего это будет сделать в два этапа, сначала разбив даанные на обучающие и валидационные, а потом отделив от обучающих тестовые.

In [8]:
features_train, features_valid, target_train, target_valid = train_test_split(features, target, test_size=0.2, random_state=12345, stratify=target)
features_train, features_test, target_train, target_test = train_test_split(features_train, target_train, test_size=0.25, random_state=12345, stratify=target_train)

Проверим правильно ли разделились выборки.

In [9]:
display(features_train.shape)
display(features_valid.shape)
display(features_test.shape)

(1928, 4)

(643, 4)

(643, 4)

Выборки готовы, значит можно переходить к оценке моделей.

## Выбор лучшей модели

Будем выбирать из трех моделей классификации:  
 - Дерево решений DecisionTreeClassifier;
 - Случайный лес RandomForestClassifier;
 - Логистическая регрессия LogisticRegression.

Начнем с дерева решений.  
Сразу рассмотрим варианты с разной глубиной(depth) и выделим наиболее удачный.

In [10]:
best_model = None
best_result = 0
best_depth = 0
for depth in range(1, 11):
    model = DecisionTreeClassifier(random_state=12345, max_depth=depth)
    model.fit(features_train, target_train)
    predictions_valid = model.predict(features_valid)
    result = accuracy_score(target_valid, predictions_valid)
    if result > best_result:
        best_model = model
        best_result = result
        best_depth = depth
display('best result', best_result)
display('best depth', best_depth)

'best result'

0.7993779160186625

'best depth'

3

Наибольший параметр accuracy показало дерево с глубиной 3.  
  
Далее рассмотрим модель случайного леса.  
Как и с деревом решений, проверим сразу несколько вариантов с разными гиперпараметрами, в данном случае это будут n-estimators(количество деревьев) и глубина(depth).

In [11]:
best_model = None
best_result = 0
best_depth = 0
for est in range(70, 130, 10):
    for depth in range (1, 11):
        model = RandomForestClassifier(random_state=12345, n_estimators=est, max_depth=depth)
        model.fit(features_train, target_train)
        predictions_valid = model.predict(features_valid)
        result = accuracy_score(target_valid, predictions_valid)
        if result > best_result:
            best_model = model
            best_result = result
            best_est = est
            best_depth = depth
display('best result', best_result)
display('best est', best_est)
display('best depth', best_depth)

'best result'

0.8211508553654744

'best est'

90

'best depth'

10

Лучший результат показал лес из 90 деревьев с глубиной 10.

И в заключение проверим логистическую регрессию.  
Тут нет особого смысла в переборе разных гиперпараметров, так что просто выставим достаточно высокий порог max-iter.

In [12]:
model = LogisticRegression(random_state=12345, solver='lbfgs', max_iter=1000)
model.fit(features_train, target_train)
predictions_valid = model.predict(features_valid)
result = accuracy_score(target_valid, predictions_valid)
result

0.7371695178849145

Логистическая регрессия показала худший результат, даже до 75% accuracy.

Делаем промежуточный вывод, что лучший параметр accuracy показала модуль случайного леса с гиперпараметрами n-estimators=31 и depth=10.    
Осталось проверить её на тестовой выборке.

## Проверка качества и вменяемости выбранной модели

Для успешного выполнения задач необходимо получить на тестовой выборке accuracy выше 75%.

In [13]:
# Дообучим модель добавив к обучающей выборке валидационную.
features_train = features_train.append(features_valid)
target_train = target_train.append(target_valid)

In [14]:
model = RandomForestClassifier(random_state=12345, n_estimators=90, max_depth=10)
model.fit(features_train, target_train)
predictions_test = model.predict(features_test)
result = accuracy_score(target_test, predictions_test)
result

0.8242612752721618

Более 82% нас полностью устраивает!  
Напоследок проверим выбранную модель на вменяемость, сравнив её результат с моделью просто заполняющей всё наиболее популярным значением.  
Такую модель получим при помощи DummyClassifier.

In [16]:
dummy_clf = DummyClassifier(strategy="most_frequent")
dummy_clf.fit(features_train, target_train)
predictions_dummy = dummy_clf.predict(features_test)
dummy_clf_result = accuracy_score(target_test, predictions_dummy)
dummy_clf_result

0.6936236391912908

Разница в 13% довольно ощутима, модель можно признать вменяемой!

## Общий вывод
Исходные данные были предоставленны в форматах приемлимых для выполнения поставленной задачи.  
Данные были разбиты на обучающую, валидационную и тестовую выборки в соотношении 3:1:1.  
Было проведено сравнения моделей дерева решений, случайного леса и логистической регрессии.  
Выбор был сделан в пользу модели случайного леса с количеством деревьев равным 21 и глубиной равной 10.  
Выбранная модель успешно прошла проверку качества и вменяемости, следовательно рекомендуется использовать именно её.